# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [301]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [302]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [303]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [304]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [305]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [306]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [307]:
# CORRECTED: Load API key from .env file
import os
import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please check your .env file.")

print("✅ API key loaded successfully from .env file!")


✅ API key loaded successfully from .env file!


### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

In [308]:
# FIXED RetrievalAugmentedQAPipeline class - handles the enhanced VectorDatabase
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, result in enumerate(context_list, 1):
            # Handle both old format (context, score) and new format (context, score, metadata)
            if len(result) == 3:
                context, score, metadata = result
            else:
                context, score = result
                metadata = None
            
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }


While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


##### ✅ Answer:
1. The "dimensions" parameter in the OpenAI API allows to change this. Supported only for text-embedding-3 and later models.
2. text-embedding-3-small was likely trained using Matryoshka Representation Learning technique to create the smaller, more efficient embedding model. MRL is a technique for training embeddings so that subsets of the embedding dimensions still retain meaningful information, much like the Matryoshka Dolls (Russian stacking dolls)

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [309]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
The "async" method works asynchronously, not blocking the thread executing it. The corresponding "sync" method blocks the thread until the embeddings are returned.

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [310]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
LLM outputs are inherently non-deterministic. But to reduce the variability, the following options exist. 
- Using a fixed "seed" parameter. Seems like this is deprecated though.
- Using the "temperature" parameter

### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [311]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [312]:
print(response)

Hello! The best way to write a loop in Python depends on what you're trying to achieve, but generally, using a `for` loop or a `while` loop is common.

Here’s a quick overview:

### Using a `for` loop
It's great when you know the number of iterations or when iterating over a sequence (like a list, tuple, string, or range).

```python
# Looping through a list
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)
```

Or with a range:

```python
# Looping a fixed number of times
for i in range(5):
    print(i)
```

### Using a `while` loop
Use this when you want to loop until a condition becomes false.

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

If you want, I’d be happy to help you write a loop for a specific task — just let me know! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [313]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [314]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [315]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or startup founder hires a weak executive to run the function or specialty that originally brought the CEO to the company, often because the CEO has a hard time letting go of that function and wants to remain in control. This results in hiring someone weak into the executive role for that function so that the CEO can continue to be “the man.” 

The problem is named after Michael Eisner, the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, the network fell to fourth place. Eisner claimed that if he had an extra two days a week, he could have turned ABC around himself, but obviously he did not have that time, illustrating the impracticality of holding onto the function oneself rather than hiring a strong executive. This concept highlights the risk of deliberately hiring a weak executive in the CEO's former specialty, resulting in organizati

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
These strategies are called Prompt Engineering, where we can guide the LLM to provide a correct, useful, detailed response. Following are some of these prompting:
- Few shot prompting - Give a couple of examples of expected output
- Chain-of-Thought prompting -  Explicitly instructing to "think step by step" where reasoning is required.
- Decomposition Prompting - Aking the model to break the problem into smaller sub-questions and solve each one before answering the main question.
- Role/Persona Prompting - Instructing model to take on a persona of an expert in a specific topic or area.



### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database
- Use a different embedding model
- Add the capability to ingest a YouTube link

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

## 🎥 YouTube RAG Enhancement

Let's enhance our RAG application to handle YouTube videos with transcripts! This will involve:

1. **YouTube Transcript Ingestion**: Extract existing transcripts from YouTube videos
2. **Enhanced Vector Database**: Store video metadata alongside embeddings  
3. **YouTube-Aware RAG Pipeline**: Query YouTube content with proper context


In [316]:
# Import the YouTube transcript functionality
from aimakerspace.text_utils import YouTubeTranscriptLoader


### Step 1: Load YouTube Transcript

Let's create a YouTube transcript loader and process a video. We'll use a video that has transcripts available.


In [317]:
# Initialize YouTube transcript loader
youtube_loader = YouTubeTranscriptLoader()

# Example YouTube URL - replace with a video that has transcripts
# For demo purposes, let's use a TED talk or educational video
youtube_url = "https://www.youtube.com/watch?v=zjkBMFhNj_g"  # Replace with actual video

print("🎥 Loading YouTube transcript...")
print(f"URL: {youtube_url}")

# Load transcript from YouTube
result = youtube_loader.load_youtube_transcript(youtube_url)

if result['success']:
    print("✅ Transcript loaded successfully!")
    print(f"📹 Video ID: {result['metadata']['video_id']}")
    print(f"📝 Transcript length: {result['metadata']['transcript_length']} characters")
    print(f"🔤 Language: {result['metadata']['language']}")
    print(f"📊 Segments: {result['metadata']['segment_count']}")
    print("\n📄 First 500 characters of transcript:")
    print(result['transcript'][:500] + "...")
else:
    print("❌ Error loading transcript:")
    print(result['error'])


🎥 Loading YouTube transcript...
URL: https://www.youtube.com/watch?v=zjkBMFhNj_g
✅ Transcript loaded successfully!
📹 Video ID: zjkBMFhNj_g
📝 Transcript length: 64352 characters
🔤 Language: en
📊 Segments: 1704

📄 First 500 characters of transcript:
hi everyone so recently I gave a 30-minute talk on large language models just kind of like an intro talk um unfortunately that talk was not recorded but a lot of people came to me after the talk and they told me that uh they really liked the talk so I would just I thought I would just re-record it and basically put it up on YouTube so here we go the busy person's intro to large language models director Scott okay so let's begin first of all what is a large language model really well a large lang...


### Step 2: Create Vector Database with YouTube Content

Now let's create a vector database that can store both the transcript text and the video metadata.


In [318]:
# Reload the VectorDatabase module to get the latest changes
import importlib
import aimakerspace.vectordatabase
importlib.reload(aimakerspace.vectordatabase)

from aimakerspace.vectordatabase import VectorDatabase


In [319]:
# Only proceed if transcript was loaded successfully
if result['success']:
    # Get document with metadata from YouTube loader
    youtube_document = youtube_loader.get_document_with_metadata()
    
    # Split the YouTube transcript into chunks
    youtube_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    youtube_chunks = youtube_splitter.split_texts([youtube_document['text']])
    
    # Create metadata for each chunk (inherit from parent document)
    youtube_chunks_with_metadata = []
    for i, chunk in enumerate(youtube_chunks):
        chunk_metadata = {
            **youtube_document['metadata'],
            'chunk_id': i,
            'chunk_length': len(chunk),
            'source_type': 'youtube_transcript'
        }
        youtube_chunks_with_metadata.append({
            'text': chunk,
            'metadata': chunk_metadata
        })
    
    print(f"📊 Created {len(youtube_chunks_with_metadata)} chunks from YouTube transcript")
    print(f"📝 Average chunk length: {sum(len(chunk['text']) for chunk in youtube_chunks_with_metadata) / len(youtube_chunks_with_metadata):.0f} characters")
    
    # Create enhanced vector database
    youtube_vector_db = VectorDatabase()
    youtube_vector_db.build_from_documents_with_metadata(youtube_chunks_with_metadata)
    
    print("✅ YouTube vector database created successfully!")
else:
    print("❌ Cannot create vector database - no transcript available")


📊 Created 161 chunks from YouTube transcript
📝 Average chunk length: 499 characters
✅ YouTube vector database created successfully!


### Step 3: Create YouTube-Aware RAG Pipeline

Let's create a specialized RAG pipeline that can handle YouTube content and provide rich context about the video.


In [320]:
# Only create RAG pipeline if we have a successful transcript
if result['success']:
    class YouTubeRAGPipeline:
        """
        Enhanced RAG pipeline specifically designed for YouTube content.
        """
        
        def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                     response_style: str = "detailed", include_scores: bool = False) -> None:
            self.llm = llm
            self.vector_db_retriever = vector_db_retriever
            self.response_style = response_style
            self.include_scores = include_scores

        def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
            # Retrieve relevant contexts with metadata
            context_list = self.vector_db_retriever.search_by_text(
                user_query, k=k, include_metadata=True
            )
            
            context_prompt = ""
            similarity_scores = []
            video_info = {}
            
            for i, (context, score, metadata) in enumerate(context_list, 1):
                context_prompt += f"[Source {i}]: {context}\n\n"
                similarity_scores.append(f"Source {i}: {score:.3f}")
                
                # Extract video information from metadata
                if metadata and 'video_id' in metadata:
                    video_info = {
                        'video_id': metadata.get('video_id', 'Unknown'),
                        'url': metadata.get('url', ''),
                        'language': metadata.get('language', 'unknown'),
                        'source_type': metadata.get('source_type', 'youtube_transcript')
                    }
            
            # Create enhanced system prompt for YouTube content
            youtube_system_template = """You are a knowledgeable assistant that answers questions based on YouTube video transcripts.

Instructions:
- Only answer questions using information from the provided video transcript
- If the transcript doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the transcript when possible
- Keep responses {response_style} and {response_length}
- When referencing the video, mention it's from a YouTube video
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

            youtube_user_template = """Video Information:
Video ID: {video_id}
Source: {source_type}
Language: {language}

Context Information from Video Transcript:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the video transcript above."""

            # Create system message
            system_params = {
                "response_style": self.response_style,
                "response_length": system_kwargs.get("response_length", "detailed")
            }
            
            system_prompt = SystemRolePrompt(youtube_system_template)
            formatted_system_prompt = system_prompt.create_message(**system_params)
            
            # Create user message with video context
            user_params = {
                "user_query": user_query,
                "context": context_prompt.strip(),
                "context_count": len(context_list),
                "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else "",
                "video_id": video_info.get('video_id', 'Unknown'),
                "source_type": video_info.get('source_type', 'YouTube transcript'),
                "language": video_info.get('language', 'unknown')
            }
            
            user_prompt = UserRolePrompt(youtube_user_template)
            formatted_user_prompt = user_prompt.create_message(**user_params)

            return {
                "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
                "context": context_list,
                "context_count": len(context_list),
                "similarity_scores": similarity_scores if self.include_scores else None,
                "video_info": video_info,
                "prompts_used": {
                    "system": formatted_system_prompt,
                    "user": formatted_user_prompt
                }
            }

    # Create YouTube RAG pipeline
    youtube_rag_pipeline = YouTubeRAGPipeline(
        vector_db_retriever=youtube_vector_db,
        llm=chat_openai,
        response_style="detailed",
        include_scores=True
    )

    print("✅ YouTube RAG pipeline created successfully!")
else:
    print("❌ Cannot create RAG pipeline - no transcript available")


✅ YouTube RAG pipeline created successfully!


### Step 4: Test the YouTube RAG Pipeline

Let's test our YouTube RAG pipeline with a sample question!


In [321]:
from IPython.display import display, Markdown

# Test the YouTube RAG pipeline if it was created successfully
if result['success'] and 'youtube_rag_pipeline' in locals():
    # Example question about the video content
    test_question = "What does the speaker say about Jailbraking?"
    
    print(f"🤔 Question: {test_question}")
    print("🔍 Searching through YouTube transcript...")
    
    # Run the RAG pipeline
    rag_result = youtube_rag_pipeline.run_pipeline(
        test_question,
        k=3,
        response_length="detailed"
    )
    
    print("\n" + "="*60)
    print("📝 ANSWER:")
    print("="*60)

    # Display the answer with markdown formatting for better readability
    display(Markdown(f"{rag_result['response']}"))

    print(f"\n📊 Context Count: {rag_result['context_count']}")
    print(f"🎯 Similarity Scores: {rag_result['similarity_scores']}")
    print(f"📹 Video Info: {rag_result['video_info']}")
else:
    print("❌ Cannot test RAG pipeline - no transcript or pipeline available")


🤔 Question: What does the speaker say about Jailbraking?
🔍 Searching through YouTube transcript...

📝 ANSWER:


Based on the YouTube video transcript, the speaker explains that jailbreaks on language models are powerful and difficult to prevent because they can bypass the model's restrictions by using cleverly crafted inputs. For example, if you ask a model like Claude directly how to do something harmful, it will refuse, but if you use a certain coded or optimized suffix of words (referred to as a "universal transferable suffix"), the model may then provide the harmful information. This suffix was discovered through an optimization process by researchers, not written by a person, and can be attached to various prompts to jailbreak the model.

The speaker also notes that jailbreaks can take many forms and combinations, making them potent attacks on large language models. Papers have studied many such jailbreak strategies, highlighting their diversity and effectiveness. Essentially, jailbreaks manipulate the model into ignoring original safeguards and producing responses that would otherwise be refused. (Sources: 1, 2, 3 from the transcript)


📊 Context Count: 3
🎯 Similarity Scores: ['Source 1: 0.489', 'Source 2: 0.458', 'Source 3: 0.433']
📹 Video Info: {'video_id': 'zjkBMFhNj_g', 'url': 'https://www.youtube.com/watch?v=zjkBMFhNj_g', 'language': 'en', 'source_type': 'youtube_transcript'}


## 🎉 YouTube RAG Enhancement Complete!

**Summary:**

✅ **YouTube Transcript Ingestion**: Created a `YouTubeTranscriptLoader` that can extract existing transcripts from YouTube videos

✅ **Enhanced Error Handling**: Proper error messages for videos without transcripts, disabled transcripts, or unavailable videos

✅ **Metadata-Rich Vector Database**: Enhanced the vector database to store video metadata alongside embeddings

✅ **YouTube-Aware RAG Pipeline**: Created a specialized RAG pipeline that provides context about the video source

✅ **Complete Integration**: Seamlessly integrated YouTube transcript processing with the existing RAG workflow

**Key Features:**
- 🎥 **Single Video Focus**: Optimized for processing one YouTube video at a time
- 🛡️ **Robust Error Handling**: Clear messages when videos cannot be processed
- 📊 **Rich Metadata**: Tracks video ID, URL, language, segment count, and more
- 🔍 **Contextual Responses**: RAG pipeline knows it's working with YouTube content
- ⚡ **Efficient Processing**: Uses existing transcripts instead of audio transcription

**To use this enhancement:**
1. Replace the example URL with a YouTube video that has transcripts
2. Run the cells to load the transcript and create the RAG pipeline
3. Ask questions about the video content using the `youtube_rag_pipeline`
